# Distributional Semantics Model: Narrow Word-Window (-2,+2)

### A. Data-Loading Facilities

In [9]:
import os
os.chdir("/Users/jacobsw/Desktop/IMPLEMENTATION_CAMP/CODE/BASIC_TOPICS/DISTRIBUTIONAL_SEMANTICS/ASSIGNMENT_03")

In [10]:
import nltk
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
from collections import defaultdict
from functools import partial
from itertools import permutations, product
punctuation = '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~'

In [11]:
import warnings
warnings.filterwarnings("ignore")

In [22]:
def load_wiki(cutoffFreq=50):
    
    print "... extracting data"
    with open('wikicorpus.txt','rb') as f:
        raw = f.readlines()
    raw = [sent[4:].split() for sent in raw if sent.startswith('<c>')] 
        # sent[4:]: get rid of initial <c>.
        # extract sentences; split sentences into word complexes.
    raw = [[map(partial(str.split, word), '|') for word in sent] for sent in raw] 
        # split word complexes into words.
    
    print "... cleaning data"
    sents = [[word[0][1].lower() for word in sent if len(word[0])>1 
              and (word[0][2].startswith('N') or word[0][2]=='.')
              and word[0][1].lower() not in stopwords
              and word[0][1] not in punctuation] for sent in raw]
        # extract lemmas => complete sents corpus .
    
    print "... building token list and vocabulary"
    tokens = [word for sent in sents for word in sent]
        # type: list of words.
    fdist = nltk.FreqDist(tokens)
    vocab = list(set(tokens))   
    
    print "... saving top %d-frequent in vocabulary" % cutoffFreq
    vocab = [word for word in vocab if fdist[word] >= cutoffFreq]
        # vocab is not returned, because the k-frequent cut latter can change it.   
    sents = [[word.decode('utf-8','ignore') for word in sent if word in vocab] for sent in sents]
        # type: list of lists of words.
    for sent in sents:
        sent.append('.')
        # append '.' to fix -1 lookahead indexing error.
        
    return sents

In [23]:
%%time
sents = load_wiki(50)

... extracting data
... cleaning data
... building token list and vocabulary
... saving top 50-frequent in vocabulary
CPU times: user 4min 58s, sys: 7.06 s, total: 5min 5s
Wall time: 5min 2s


### B1. Analyzer: Cooccurrence Matrix Based

#### A. Model

In [25]:
import numpy as np

In [42]:
# SIMILARITY MEASURES
def cosine(w2w):
    w2w_norm = w2w / np.apply_along_axis(lambda r: np.sqrt(np.dot(r,r))
                               , 1, w2w)[:,np.newaxis]
    return np.dot(w2w_norm, w2w_norm.T)
    
def ppmi(w2w):
    rowSums, colSums, totalSums = w2w.sum(axis=1), w2w.sum(axis=0), w2w.sum()
    pwi, pwj, ppmiMatrix = rowSums/totalSums, colSums/totalSums, w2w/totalSums
    ppmiMatrix /= pwi[:,np.newaxis] # * 1/pwi by row.
    ppmiMatrix /= pwj # * 1/pwj by col.
    ppmiMatrix = np.nan_to_num(np.log(ppmiMatrix)) # compute pmi.
    ppmiMatrix = np.maximum(ppmiMatrix, 0) # compute ppmi.
    return ppmiMatrix

In [43]:
class SimpleDistSem:
    
    def __init__(self, data=sents):
        self.sents = sents 
        self.vocab = list({word for sent in self.sents for word in sent})
        self.wordToIndex = {word:i for i,word in enumerate(self.vocab)}
        self.indexToWord = {i:word for word,i in self.wordToIndex.iteritems()}      

    def build_w2w_matrix(self, window=2):
        
        print "... counting words"
        cooccurrenceDict = defaultdict(int)
        def add_entry(word,contexts):
            for context in contexts:
                cooccurrenceDict[(self.wordToIndex[word],self.wordToIndex[context])] += 1
        for sent in self.sents:
            for i,word in enumerate(sent):
                if word=='.': continue # not pass, because we do not execute anything after this.
                contexts = sent[max(0,i-window):i] + sent[min(len(sent)-1,i+1):min(len(sent),i+window+1)]
                add_entry(word,contexts)                
        
        print "... building cooccurrence matrix"
        self.w2w = np.zeros((len(self.vocab),len(self.vocab)))
        for (widx_i,widx_j),count in cooccurrenceDict.iteritems():
            self.w2w[widx_i][widx_j] = count   
    
    def build_similarity_matrix(self, similarity=ppmi):
        self.simMatrix = similarity(self.w2w)
    
    def k_most_similar(self, words, k=20):
        assert len(words)==len(filter(lambda w:1 if w in self.vocab else 0, words))
        w2sim = {}
        for word in words:
            simList = self.simMatrix[self.wordToIndex[word]]
            w2sim[word] = map(lambda idx:(self.indexToWord[idx],
                                          self.simMatrix[self.wordToIndex[word]][idx]),
                              np.argsort(simList)[::-1][1:k+1])
            # [1:k+1]: skip self-similarty.
        return w2sim

In [45]:
%%time
ds = SimpleDistSem(sents)

CPU times: user 430 ms, sys: 11.7 ms, total: 441 ms
Wall time: 433 ms


In [46]:
%%time
ds.build_w2w_matrix()

... counting words
... building cooccurrence matrix
CPU times: user 12.1 s, sys: 425 ms, total: 12.5 s
Wall time: 12.5 s


#### b. Evaluator: K-Frequent

In [47]:
%%time
ds.build_similarity_matrix(ppmi)

CPU times: user 1.73 s, sys: 660 ms, total: 2.4 s
Wall time: 2.41 s


In [48]:
%%time
words = ['car','bus','hospital','hotel','gun','bomb','horse','fox','table','bowl','guitar','piano']
w2sim = ds.k_most_similar(words)

CPU times: user 9.35 ms, sys: 820 µs, total: 10.2 ms
Wall time: 9.18 ms


In [49]:
w2sim['car']

[(u'bogie', 4.7499189450703065),
 (u'brabham', 4.7423143456850871),
 (u'racing', 4.617876563176341),
 (u'brake', 4.3551983762451192),
 (u'bentley', 4.3551983762451192),
 (u'earnhardt', 4.3456289252289695),
 (u'cable', 4.3248730465302074),
 (u'clutch', 4.3033265455382796),
 (u'audi', 4.1833481193184605),
 (u'chassis', 4.1486527163399476),
 (u'aston', 4.0823832151336719),
 (u'truck', 4.0752213501677437),
 (u'grip', 3.9971355070315391),
 (u'car', 3.9722343921407028),
 (u'installation', 3.8952484231395825),
 (u'tire', 3.8869323668979252),
 (u'motors', 3.8725436294458255),
 (u'prix', 3.8260236138109325),
 (u'crash', 3.7903055312088534),
 (u'luxury', 3.7739919557173294)]

In [50]:
%%time
ds.build_similarity_matrix(cosine)

CPU times: user 21.9 s, sys: 376 ms, total: 22.3 s
Wall time: 3.94 s


In [51]:
%%time
words = ['car','bus','hospital','hotel','gun','bomb','horse','fox','table','bowl','guitar','piano']
w2sim = ds.k_most_similar(words)

CPU times: user 11.6 ms, sys: 681 µs, total: 12.3 ms
Wall time: 11.4 ms


In [52]:
w2sim['piano']

[(u'piano', 1.0),
 (u'cello', 0.90165413185944943),
 (u'instrument', 0.90150679400641198),
 (u'piece', 0.89701239728546034),
 (u'concerto', 0.8903376569080329),
 (u'style', 0.88796392706787153),
 (u'composition', 0.88761026953361855),
 (u'music', 0.88691920462707574),
 (u'sound', 0.88495449558076045),
 (u'work', 0.88091736205823024),
 (u'arrangement', 0.88073497919205657),
 (u'performance', 0.87848388859395399),
 (u'career', 0.87573004350436723),
 (u'theme', 0.87502686110213812),
 (u'accompaniment', 0.8742842987954812),
 (u'audience', 0.87422318981203706),
 (u'influence', 0.87351512481564564),
 (u'tone', 0.87331561279428938),
 (u'guitar', 0.87327242099322699),
 (u'history', 0.87299835537705384)]

#### c. Evaluator: BLESS

In [53]:
import random

In [54]:
def bless_evaluator(simMatrix=None, indexers=[None,None]):
    wordToIndex, indexToWord = indexers
    path = '/Users/jacobsw/Desktop/IMPLEMENTATION_CAMP/CODE/BASIC_TOPICS/DISTRIBUTIONAL_SEMANTICS/ASSIGNMENT_03/BLESS_part.txt'
    with open(path,'rb') as f:
        bless = f.readlines()
    bless = [line.split('\t') for line in bless] # split into (concept, _, relation, relatum).
    crPairs = [(c.split('-')[0],r.split('-')[0],rel) for c,_,rel,r in bless]
    posPairs = [(c,r) for c,r,rel in crPairs if rel=='hyper']
    negPairs = [(c,r) for c,r,rel in crPairs if rel=='mero']
    
    return [map(lambda (c,r):(c,r,simMatrix[wordToIndex[c]][wordToIndex[r]]), posPairs),
            map(lambda (c,r):(c,r,simMatrix[wordToIndex[c]][wordToIndex[r]]), negPairs)]

##### PPMI

In [55]:
%%time
ds.build_similarity_matrix(ppmi)
posEval, negEval = bless_evaluator(ds.simMatrix, indexers=[ds.wordToIndex, ds.indexToWord])

CPU times: user 1.79 s, sys: 675 ms, total: 2.47 s
Wall time: 2.48 s


In [56]:
print "Examples of Evaluation on Positive Relations (PPMI): "
print random.sample(posEval, 5)
print "Average PPMI: ", np.mean([ppmiVal for _,_,ppmiVal in posEval])

Examples of Evaluation on Positive Relations (PPMI): 
[('glove', 'commodity', 0.0), ('cranberry', 'produce', 0.0), ('fighter', 'craft', 2.4391216948827172), ('phone', 'good', 0.0), ('car', 'vehicle', 2.8610466192688153)]
Average PPMI:  1.49359812577


In [57]:
print "Examples of Evaluation on Negative Relations (PPMI): "
print random.sample(negEval, 5)
print "Average PPMI: ", np.mean([ppmiVal for _,_,ppmiVal in negEval])

Examples of Evaluation on Negative Relations (PPMI): 
[('potato', 'skin', 2.8919126085645726), ('fox', 'nose', 0.0), ('library', 'ladder', 0.0), ('table', 'leg', 0.70001437753646556), ('falcon', 'eye', 0.0)]
Average PPMI:  1.295598781


##### Cosine

In [58]:
%%time
ds.build_similarity_matrix(cosine)
posEval, negEval = bless_evaluator(ds.simMatrix, indexers=[ds.wordToIndex, ds.indexToWord])

CPU times: user 22.9 s, sys: 399 ms, total: 23.3 s
Wall time: 4.26 s


In [59]:
print "Examples of Evaluation on Positive Relations (Cosine): "
print random.sample(posEval, 5)
print "Average Cosine: ", np.mean([cosineVal for _,_,cosineVal in posEval])

Examples of Evaluation on Positive Relations (Cosine): 
[('rifle', 'arm', 0.86255685115276515), ('elephant', 'vertebrate', 0.89362826693552888), ('snake', 'reptile', 0.737574716119763), ('grape', 'food', 0.74779136683837233), ('missile', 'vehicle', 0.86425198715387352)]
Average Cosine:  0.837919844858


In [60]:
print "Examples of Evaluation on Positive Relations (Cosine): "
print random.sample(negEval, 5)
print "Average Cosine: ", np.mean([cosineVal for _,_,cosineVal in negEval])

Examples of Evaluation on Positive Relations (Cosine): 
[('violin', 'scroll', 0.65415068794244524), ('horse', 'hair', 0.91438607723597176), ('hotel', 'stone', 0.89319380418002292), ('castle', 'queen', 0.84488075612036417), ('pub', 'seat', 0.79518812164472408)]
Average Cosine:  0.815547782333
